## Final Project Submission

Please fill out:
* Student name: Lameck Odallo
* Student pace: part time 
* Scheduled project review date/time: 27th July 1200
* Instructor name: Christine
* Blog post URL:


In [6]:
# Importing all the relevant packages (Including statistical ones)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette="Set1")
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
%matplotlib inline

In [7]:
#importing the dataset 
df = pd.read_csv('data/Aviation_Data.csv', low_memory=False)

First Step was to explore the data to check the structure and completeness of data

Conclusion: The dataset contains over 90,000 records and more than 30 variables. Most of the data is in text format, with a few numeric columns. While some fields are mostly complete, others have significant missing data. There’s enough clean and relevant information to support meaningful analysis — particularly the injury counts, aircraft damage classification, and severity of events. Some data points may need to be dropped, but with 90+K observations this may not be a huge risk. 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

To determine the airplanes with the lowest risk, I started by prioritizing the identification variables for the airplanes, which without we can't really make any logical conclusions. The two variables are 'make'and 'model'

In [9]:
#exploring the plane identification variables - make
df['Make'].value_counts(dropna=False)


Cessna                 22227
Piper                  12029
CESSNA                  4922
Beech                   4330
PIPER                   2841
                       ...  
Randy Wayne Maloney        1
Bonds & Bungey             1
LANTZAIR FLYERS INC        1
Mcmillin                   1
Ralph C Kimball            1
Name: Make, Length: 8238, dtype: int64

In [10]:
#exploring the plane identification variables - model
df['Model'].value_counts(dropna=False)

152              2367
172              1756
NaN              1551
172N             1164
PA-28-140         932
                 ... 
CLOWN               1
47G (TOM CAT)       1
CHAMPION 7BCM       1
DHC-2-L-20          1
767-3W0ER           1
Name: Model, Length: 12319, dtype: int64

The model variable does not seem as useful for now, but let's see if we can use it for any imputation later

In [11]:
#clean up of the 'make'variable
## since there is a mix of upper and lower case for the values contained in the 'make variable'
### first step is to sentence-case the 'make'variable as i'll be using it in visualizations (normalizing)
df['Make'] = df['Make'].str.strip().str.title()

In [12]:
df['Make'].value_counts(dropna=False)

Cessna                27149
Piper                 14870
Beech                  5372
Boeing                 2745
Bell                   2722
                      ...  
Buhlmann Howard           1
Hunziker                  1
Eric A Munzer             1
Loofbourrow John W        1
Baby Bird                 1
Name: Make, Length: 7588, dtype: int64

In [13]:
#Note that the code below creates a long t
#makes_list = sorted(df['Make'].dropna().unique().tolist())
#for make in makes_list:
 #   print(make)

From the above code (now muted, as it creates a long list), I realized that see that there are string values that were very closely related such as Aero Tek Inc.
Aero Tek, Inc.
Aero Vodochody
Aero Vodochody Aero Works
Aero Vodochody Aero. Works
which prompted me to consider fuzzy matching but in a new variable, so that the original variable remains unchanged

In [14]:
#let us first see what the top 50 makes are - this is arbitrary but it gives us a good indicator of the most used flights and what the company would consider
top_makes = df['Make'].value_counts().head(50)
top_makes.to_frame(name="Frequency")

,Frequency
Cessna,27149
Piper,14870
Beech,5372
Boeing,2745
Bell,2722
Mooney,1334
Robinson,1230
Grumman,1172
Bellanca,1045
Hughes,932


In [15]:
pip install 'thefuzz[speedup]'

Note: you may need to restart the kernel to use updated packages.


In [16]:
#implementing the fuzzy match process

from thefuzz import process

# Step 1: Convert the top_makes DataFrame to a list 
top_50_list = top_makes.index.tolist()  

# Step 2: Define the fuzzy match function
def match_to_top_50(make):
    if pd.isna(make):
        return None  
    match, score = process.extractOne(make, top_50_list)
    if score >= 85:
        return match  
    else:
        return None   

# Step 3: Apply the match function 
df['Matched_Make'] = df['Make'].apply(match_to_top_50)

df[['Make', 'Matched_Make']].sample(10)  


,Make,Matched_Make
56918,Cessna,Cessna
25882,Cessna,Cessna
48270,Bell,Bell
52440,Grote,None
8743,Beech,Beech
24867,Piper,Piper
37240,Curtiss-Wright,None
41489,Fokker,None
41885,Giertz,None
42736,Cessna,Cessna


In [21]:
#exporting the data set to see how the fuzzy match worked
df[['Make', 'Matched_Make']].to_csv("all_make_matches.csv", index=False)

In [22]:
#this is a variable of interest, we can use it a control variable 
df['Amateur.Built'].unique()

array(['No', 'Yes', nan], dtype=object)

In [27]:
#df['Number.of.Engines'].describe()  
df['Purpose.of.flight'].value_counts()

Personal                     49448
Instructional                10601
Unknown                       6802
Aerial Application            4712
Business                      4018
Positioning                   1646
Other Work Use                1264
Ferry                          812
Aerial Observation             794
Public Aircraft                720
Executive/corporate            553
Flight Test                    405
Skydiving                      182
External Load                  123
Public Aircraft - Federal      105
Banner Tow                     101
Air Race show                   99
Public Aircraft - Local         74
Public Aircraft - State         64
Air Race/show                   59
Glider Tow                      53
Firefighting                    40
Air Drop                        11
ASHO                             6
PUBS                             4
PUBL                             1
Name: Purpose.of.flight, dtype: int64

In [28]:
#df['Air.carrier'].unique()

array([nan, 'Air Canada', 'Rocky Mountain Helicopters, In', ...,
       'SKY WEST AVIATION INC TRUSTEE', 'GERBER RICHARD E',
       'MC CESSNA 210N LLC'], dtype=object)

In [31]:
df[['Total.Fatal.Injuries','Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].describe()

,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,77488.000000,76379.000000,76956.000000,82977.000000
mean,0.647855,0.279881,0.357061,5.325440
std,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,2.000000
max,349.000000,161.000000,380.000000,699.000000


In [32]:
df['Weather.Condition'].unique()

array(['UNK', 'IMC', 'VMC', nan, 'Unk'], dtype=object)